In [13]:
import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
from nba_api.stats.endpoints import leaguegamefinder
import psycopg2
import re
import allintwo

### 
testplayer='203076'
testgame = '0022201200'
testteam = '1610612755'

In [23]:

conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')
    
query = "SELECT * FROM pg_catalog.pg_tables where tableowner = 'ajwin';"  # Example query: fetch 10 rows from the 'games' table

if conn:
    result_df = allintwo.query_database_to_dataframe(conn, query)

conn.close()



Connected to RDS PostgreSQL database


C:\Users\ajwin\AppData\Local\Temp\ipykernel_98260\3066269946.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = pd.read_sql(query, conn)


In [21]:
### CHECK TABLES

cursor = conn.cursor()

cursor.execute("""
        SELECT tablename
        FROM pg_tables
        WHERE schemaname = 'public';
    """)


cursor.fetchall()

[('masterplayers',), ('mastergames',)]

In [19]:
### DROP TABLES
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS playerids;")
conn.commit()

In [6]:
## MASTER CREATION, should complete this first





In [67]:

### TO BE INCLUDED IN THE ARGUMENT param (team or player or game (or any combination)) , endpoint

## adds the connection
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

## Naming stuff
index = 0
param = testgame

# need to replace with the endpoint class
namepre = nbaapi.boxscoreadvancedv3.BoxScoreAdvancedV3.__name__.lower()
name = namepre+"_"+str(index)

df = nbaapi.boxscoreadvancedv3.BoxScoreAdvancedV3(param).get_data_frames()[index]

df = allintwo.clean_column_names(df)


if not allintwo.check_table_exists(conn,name):
    allintwo.create_table(conn,name,df)
    ## Need to put the back fill process here

tdf = allintwo.fetch_table_data(conn,name)

gdf = allintwo.fetch_table_data(conn,"games")







Connected to RDS PostgreSQL database
Master table boxscoreadvancedv3 created successfully.


In [60]:
allintwo.get_api_data(nbaapi.boxscoreadvancedv2.BoxScoreAdvancedV2,1,testgame)

,GAME_ID,TEAM_ID,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CITY,MIN,E_OFF_RATING,OFF_RATING,E_DEF_RATING,DEF_RATING,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0022201200,1610612762,Jazz,UTA,Utah,240.000000:00,96.0,95.1,109.2,109.6,...,18.4,0.524,0.544,1.0,0.200,103.24,103.5,86.25,103,0.435
1,0022201200,1610612760,Thunder,OKC,Oklahoma City,240.000000:00,109.2,109.6,96.0,95.1,...,12.5,0.479,0.526,1.0,0.201,103.24,103.5,86.25,104,0.565


In [5]:
index = 0

param = testgame

namepre = nbaapi.boxscoreadvancedv3.BoxScoreAdvancedV3.__name__.lower()

name = namepre+"_"+str(index)

df = nbaapi.boxscoreadvancedv3.BoxScoreAdvancedV3(param).get_data_frames()[index]

name

'boxscoreadvancedv3_0'

In [58]:
gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_type_nullable="Regular Season",season_nullable='2022-23').get_data_frames()[0]
gamefinder

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1610612745,HOU,Houston Rockets,0022201222,2023-04-09,HOU @ WAS,W,241,114,...,0.714,13,38,51,24,8,6,15,15,5.0
1,22022,1610612758,SAC,Sacramento Kings,0022201227,2023-04-09,SAC @ DEN,L,241,95,...,0.667,10,24,34,24,7,6,15,18,-14.0
2,22022,1610612765,DET,Detroit Pistons,0022201223,2023-04-09,DET @ CHI,L,240,81,...,0.769,15,44,59,20,1,3,24,16,-22.0
3,22022,1610612738,BOS,Boston Celtics,0022201216,2023-04-09,BOS vs. ATL,W,240,120,...,0.786,12,35,47,33,5,3,15,13,6.0
4,22022,1610612760,OKC,Oklahoma City Thunder,0022201226,2023-04-09,OKC vs. MEM,W,240,115,...,0.813,8,41,49,30,5,2,14,17,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,22022,1610612737,ATL,Atlanta Hawks,0022200005,2022-10-19,ATL vs. HOU,W,240,117,...,0.833,4,34,38,30,12,5,9,18,10.0
2456,22022,1610612744,GSW,Golden State Warriors,0022200002,2022-10-18,GSW vs. LAL,W,241,123,...,0.739,11,37,48,31,11,4,18,23,14.0
2457,22022,1610612755,PHI,Philadelphia 76ers,0022200001,2022-10-18,PHI @ BOS,L,239,117,...,0.857,4,27,31,16,8,3,14,25,-9.0
2458,22022,1610612738,BOS,Boston Celtics,0022200001,2022-10-18,BOS vs. PHI,W,240,126,...,0.786,6,30,36,24,8,3,10,24,9.0


In [ ]:

pd.set_option('display.max_columns', 500)

seasons = []
for i in list(range(2020,2024)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]

gamesl = []
for i in seasons:
    gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_type_nullable="Regular Season",season_nullable=i).get_data_frames()[0]
    gamesl.append(gamefinder)
    time.sleep(1)
gamehistory = pd.concat(gamesl,axis=0)
gamehistory['GAME_DATE'] =  pd.to_datetime(gamehistory['GAME_DATE'])

playersl = []
for i in seasons:
    playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=i).get_data_frames()[0]
    playerfinder['season']=i
    playersl.append(playerfinder)
    time.sleep(.5)
allplayers = pd.concat(playersl,axis=0)

In [24]:
leaguegamefinder.LeagueGameFinder(date_to_nullable='10/28/2023',date_from_nullable='10/28/2023').get_data_frames()[0]


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22023,1610612756,PHX,Phoenix Suns,0022300094,2023-10-28,PHX vs. UTA,W,241,126,45,85,0.529,10,37,0.270,26,28,0.929,7,32,39,31,10,7,11,19,22.0
1,22023,1610612755,PHI,Philadelphia 76ers,0022300092,2023-10-28,PHI @ TOR,W,241,114,43,86,0.500,10,28,0.357,18,21,0.857,9,33,42,25,8,7,9,18,7.0
2,22023,1610612763,MEM,Memphis Grizzlies,0022300089,2023-10-28,MEM @ WAS,L,240,106,39,91,0.429,19,47,0.404,9,9,1.000,6,39,45,27,7,8,16,21,-7.0
3,22023,1610612741,CHI,Chicago Bulls,0022300088,2023-10-28,CHI @ DET,L,239,102,39,93,0.419,11,28,0.393,13,16,0.813,13,20,33,16,10,5,8,20,-16.0
4,22023,1610612764,WAS,Washington Wizards,0022300089,2023-10-28,WAS vs. MEM,W,240,113,40,93,0.430,16,46,0.348,17,22,0.773,11,34,45,24,9,5,11,15,7.0
5,22023,1610612748,MIA,Miami Heat,0022300093,2023-10-28,MIA @ MIN,L,240,90,35,87,0.402,12,38,0.316,8,14,0.571,9,33,42,24,6,1,14,12,-16.0
6,22023,1610612761,TOR,Toronto Raptors,0022300092,2023-10-28,TOR vs. PHI,L,240,107,38,87,0.437,17,32,0.531,14,19,0.737,13,28,41,28,5,5,13,19,-7.0
7,22023,1610612740,NOP,New Orleans Pelicans,0022300090,2023-10-28,NOP vs. NYK,W,241,96,40,87,0.460,9,33,0.273,7,16,0.438,4,39,43,20,8,5,13,22,9.0
8,22023,1610612752,NYK,New York Knicks,0022300090,2023-10-28,NYK @ NOP,L,242,87,33,90,0.367,7,37,0.189,14,18,0.778,16,43,59,19,7,3,18,17,-9.0
9,22023,1610612765,DET,Detroit Pistons,0022300088,2023-10-28,DET vs. CHI,W,242,118,45,86,0.523,12,28,0.429,16,20,0.800,15,38,53,29,4,4,15,18,16.0


In [42]:
nbaapi.playergamelogs.PlayerGameLogs(date_from_nullable='10/20/2000',
    date_to_nullable='12/31/2001',  
    season_nullable='2000-01').get_data_frames()[0].sort_values('GAME_DATE')

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,AVAILABLE_FLAG,MIN_SEC
23972,2000-01,251,Sean Elliott,Sean,1610612759,SAS,San Antonio Spurs,0020000009,2000-10-31T00:00:00,SAS vs. IND,W,25.645000,3,7,0.429,2,4,0.500,2,2,1.000,0,0,0,2,0,0,0,0,4,0,10,11,13.0,0,0,14.0,1,1,1,1,11048,10068,10857,11237,1254,2058,2291,6583,8212,1,13595,19944,21122,7623,16423,11514,7539,1,18667,165,8809,2992,15064,2081,46,13486,None,25:39
23793,2000-01,673,Alan Henderson,Alan,1610612737,ATL,Atlanta Hawks,0020000004,2000-10-31T00:00:00,ATL vs. CHH,L,35.900000,9,11,0.818,0,0,0.000,5,9,0.556,1,1,2,2,4,3,1,0,4,0,23,-12,36.4,0,0,35.0,1,11963,11963,11963,4998,1294,5916,1253,6642,11355,6642,2061,872,10196,7381,15611,14531,7623,900,555,2810,1,18667,165,1611,21136,3427,2081,46,3002,None,35:54
23792,2000-01,452,Vin Baker,Vin,1610612760,SEA,Seattle SuperSonics,0020000013,2000-10-31T00:00:00,SEA @ VAN,L,37.473333,7,16,0.438,0,0,0.000,5,6,0.833,2,5,7,1,3,1,4,1,3,0,19,-3,40.9,0,0,37.0,1,11963,11963,11963,4069,2828,2449,11104,6642,11355,6642,2061,2259,6470,3870,3997,4027,11220,2182,4616,257,15185,14225,165,2930,14292,2321,2081,46,2491,None,37:28
23791,2000-01,764,David Robinson,David,1610612759,SAS,San Antonio Spurs,0020000009,2000-10-31T00:00:00,SAS vs. IND,W,31.111667,5,9,0.556,0,0,0.000,12,12,1.000,1,8,9,0,1,1,3,1,2,0,22,18,43.8,0,0,39.0,1,1,1,1,7743,5569,8143,5547,6642,11355,6642,99,279,1,7381,1166,2244,16481,9383,4616,543,15185,8838,165,1863,1031,1780,2081,46,2054,None,31:07
23790,2000-01,1517,Bobby Jackson,Bobby,1610612758,SAC,Sacramento Kings,0020000006,2000-10-31T00:00:00,SAC @ CHI,W,36.233333,5,9,0.556,1,3,0.333,3,3,1.000,0,4,4,6,1,4,0,1,4,0,14,18,38.8,0,0,33.0,1,1,1,1,4795,5569,8143,5547,2963,3416,4566,4676,7041,1,13595,5746,9016,1727,9383,172,7539,15185,18667,165,5590,1031,2829,2081,46,3606,None,36:14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,2000-01,213,Antonio Davis,Antonio,1610612761,TOR,Toronto Raptors,0020001182,2001-04-18T00:00:00,TOR @ WAS,W,31.468333,5,12,0.417,0,0,0.000,0,0,0.000,2,8,10,1,1,1,0,1,3,0,10,5,25.5,1,0,23.0,1,1,1,1,7541,5569,5057,11927,6642,11355,6642,13632,14518,13632,3870,1166,1630,11220,9383,4616,7539,15185,14225,165,8809,6400,7741,1,46,7894,None,31:28
142,2000-01,1497,Chauncey Billups,Chauncey,1610612750,MIN,Minnesota Timberwolves,0020001185,2001-04-18T00:00:00,MIN @ DAL,L,19.250000,8,13,0.615,3,4,0.750,2,2,1.000,0,1,1,2,0,1,0,1,1,0,21,13,28.2,0,0,29.0,1,11963,11963,11963,15112,1945,4248,3880,451,2058,1303,6583,8212,1,13595,15611,17898,7623,16423,4616,7539,15185,3659,165,2205,2218,6459,2081,46,5050,None,19:15
141,2000-01,702,Voshon Lenard,Voshon,1610612743,DEN,Denver Nuggets,0020001186,2001-04-18T00:00:00,DEN vs. SAC,W,27.966667,5,14,0.357,3,7,0.429,0,0,0.000,0,4,4,7,3,1,0,0,2,0,13,13,28.3,0,0,29.0,1,1,1,1,9607,5569,3540,14402,451,395,4073,13632,14518,13632,13595,5746,9016,1164,2182,4616,7539,1,8838,165,6325,2218,6408,2081,46,5050,None,27:58
152,2000-01,239,Darrick Martin,Darrick,1610612758,SAC,Sacramento Kings,0020001186,2001-04-18T00:00:00,SAC @ DEN,L,21.033333,4,8,0.500,2,3,0.667,5,6,0.833,0,3,3,3,1,0,0,1,3,0,15,0,22.1,0,0,23.0,1,11963,11963,11963,14002,7605,9408,6674,1254,3416,1504,2061,2259,6470,13595,8242,11534,5270,9383,11514,7539,15185,14225,165,4959,10979,9501,2081,46,789

In [1]:
import pandas as pd
import psycopg2

def fetch_table_to_dataframe(conn, table_name):

    # Create a cursor object
    cur = conn.cursor()
    
    try:
        # Execute the SQL query to fetch all data from the table
        query = f"SELECT * FROM {table_name};"
        cur.execute(query)
        
        # Fetch all the data
        data = cur.fetchall()
        
        # Get the column names from the cursor
        colnames = [desc[0] for desc in cur.description]
        
        # Convert the data into a pandas DataFrame
        df = pd.DataFrame(data, columns=colnames)
        
        print(f"Data fetched successfully from {table_name} table.")
        return df
    
    except Exception as e:
        print(f"Error: {e}")
        return None



# Call the function, passing the existing connection
df = fetch_table_to_dataframe(conn, 'mastergames')

# Display the DataFrame
df


NameError: name 'conn' is not defined

: 